In [1]:
import json
import os
from pathlib import Path

Read NPZ files

In [ ]:

from numpy import load

data = load('../data/processed/hpo2hpo.npz')
lst = data.files
for item in lst:
    print(item)
    print(data[item])

Save HPO embeddings to TXT file

In [3]:
import pickle
import torch  # If using PyTorch tensors
import numpy as np  # If using NumPy arrays

# Load the pickle file
pkl_path = "/home/abamini/PhenoGnet/Code/wandb/latest-run/files/gene_embedding.pkl"
txt_path = "/home/abamini/PhenoGnet/Code/wandb/latest-run/files/gene_embedding.txt"

with open(pkl_path, "rb") as pkl_file:
    data = pickle.load(pkl_file)  # Load the tensor data

# Ensure it's a NumPy array for easier handling
if isinstance(data, torch.Tensor):
    data = data.numpy()  # Convert PyTorch tensor to NumPy array

elif not isinstance(data, np.ndarray):
    raise TypeError("Loaded data is not a tensor or NumPy array.")

# Save embeddings line by line
with open(txt_path, "w") as txt_file:
    for embedding in data:
        txt_file.write(" ".join(map(str, embedding)) + "\n")  # Save each embedding on a new line

print(f"Embeddings saved to {txt_path}")


Embeddings saved to /home/abamini/PhenoGnet/Code/wandb/latest-run/files/gene_embedding.txt


Evaluate Embeddings !!

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

pkl_path = "/home/abamini/PhenoGnet/Code/wandb/latest-run/files/gene_embedding.pkl"
hpo2id_path = "/home/abamini/PhenoGnet/data/processed/hpo2id.txt"
file_similar_path = "/home/abamini/PhenoGnet/data/processed/positive_hpo_terms.csv"
file_random_path = "/home/abamini/PhenoGnet/data/processed/random_hpo_terms_1.csv"


def load_embeddings(pkl_path):
    """Load embeddings from a pickle file."""
    with open(pkl_path, "rb") as f:
        return pickle.load(f)  # Assuming it's a NumPy array

def load_hpo_to_id(hpo2id_path):
    """Load HPO-to-ID mapping from a text file."""
    hpo2id = {}
    with open(hpo2id_path, "r") as f:
        next(f)  # Skip the first line
        for line in f:
            hpo_id, idx = line.strip().split("\t")
            hpo2id[hpo_id] = int(idx)
    return hpo2id

def compute_avg_cosine_similarity(file_path, embeddings, hpo2id):
    """Compute average cosine similarity for given HPO term pairs."""
    similarities = []
    with open(file_path, "r") as f:
        next(f, None)  # Skip header if present
        for line in f:
            hpo1, hpo2 = line.strip().split(",")
            if hpo1 in hpo2id and hpo2 in hpo2id:
                vec1 = embeddings[hpo2id[hpo1]].reshape(1, -1)
                vec2 = embeddings[hpo2id[hpo2]].reshape(1, -1)
                sim = cosine_similarity(vec1, vec2)[0, 0]
                similarities.append(sim)
    return np.mean(similarities) if similarities else 0

def plot_similarity(avg_similar, avg_random):
    """Plot the average cosine similarity of similar and random HPO term pairs."""
    labels = ["Similar Pairs", "Random Pairs"]
    values = [avg_similar, avg_random]
    
    plt.figure(figsize=(6, 4))
    plt.bar(labels, values, color=["blue", "red"])
    plt.ylabel("Average Cosine Similarity")
    plt.title("Average Cosine Similarity of HPO Term Pairs")
    plt.ylim(0, 1)
    plt.show()



# Load data
embeddings = load_embeddings(pkl_path)
hpo2id = load_hpo_to_id(hpo2id_path)

# Compute similarities
avg_similar = compute_avg_cosine_similarity(file_similar_path, embeddings, hpo2id)
avg_random = compute_avg_cosine_similarity(file_random_path, embeddings, hpo2id)

# Plot results
plot_similarity(avg_similar, avg_random)

ValueError: not enough values to unpack (expected 2, got 1)